In [1]:
dim_reduction = [TruncatedSVD, PCA]
components = [5, 10, 20, 50, 100, 300]
dim_red_comps = [(i, j) for i in dim_reduction for j in components]

NameError: name 'TruncatedSVD' is not defined

In [ ]:
pca = PCA(n_components=5)
msg_embs_train_reduct = pd.DataFrame(pca.fit_transform(msg_embs_train))
msg_embs_test_reduct = pd.DataFrame(pca.transform(msg_embs_test))

msg_embs_train_reduct.index, msg_embs_test_reduct.index = train_idxs, test_idxs

In [ ]:
X_train_emb = pd.concat([X_train, msg_embs_train_reduct], axis=1)
X_test_emb = pd.concat([X_test, msg_embs_test_reduct], axis=1)

In [ ]:
for model in zoo_models:
    print(f"Модель - {model}")

    for red, comp in dim_red_comps:
        print(f"{red}________{comp}")
        red_model = red(n_components=comp)
        msg_embs_train_reduct = pd.DataFrame(pca.fit_transform(msg_embs_train))
        msg_embs_test_reduct = pd.DataFrame(pca.transform(msg_embs_test))
        msg_embs_train_reduct.index, msg_embs_test_reduct.index = train_idxs, test_idxs
        X_train_emb = pd.concat([X_train, msg_embs_train_reduct], axis=1)
        X_test_emb = pd.concat([X_test, msg_embs_test_reduct], axis=1)
    
        model.fit(X_train_emb.to_numpy(), y_train.to_numpy())
        preds = model.predict(X_test_emb.to_numpy())
        print(f"Среднее квадратное отклонение: {mean_squared_error(y_test, preds)}")
        # print(f"Среднее абсолютное отклонение: {mean_absolute_error(y_test, preds)}")
    print("_________________________________________________")